In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-07-31 19:58:30--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving devproxy.bloomberg.com (devproxy.bloomberg.com)... 69.191.253.138, 69.187.22.138
Connecting to devproxy.bloomberg.com (devproxy.bloomberg.com)|69.191.253.138|:82... connected.
Proxy request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'input.txt'

100%[======================================>] 1,115,394   --.-K/s   in 0.1s    

2023-07-31 19:58:30 (9.80 MB/s) - 'input.txt' saved [1115394/1115394]



In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for i,c in enumerate(chars)}
encode = lambda line : [stoi[c] for c in line]
decode = lambda encoded : ''.join([itos[i] for i in encoded])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [10]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [11]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [13]:
# block_size is the sequence size, not the batch size.
block_size = 8
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47])

In [14]:
# This way of splitting is not just for training efficiency.
# But also make Transformer get used to predicting using difference lengths of context.
x = train_data[ : block_size]
y = train_data[1 : block_size + 1]
for i in range(block_size):
    print(f'x = {x[ : i + 1]}, y = {y[i]}')

x = tensor([18]), y = 47
x = tensor([18, 47]), y = 56
x = tensor([18, 47, 56]), y = 57
x = tensor([18, 47, 56, 57]), y = 58
x = tensor([18, 47, 56, 57, 58]), y = 1
x = tensor([18, 47, 56, 57, 58,  1]), y = 15
x = tensor([18, 47, 56, 57, 58,  1, 15]), y = 47
x = tensor([18, 47, 56, 57, 58,  1, 15, 47]), y = 58


In [23]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split : str) :
    data = train_data if split == 'train' else val_data
    indices = torch.randint(len(data) - block_size, (batch_size, ))
    # Should actually be this commented out one; The original code above may out of bound for yb
    indices = torch.randint(len(data) - block_size - 1, (batch_size, ))
    xb = torch.stack([data[i: i + block_size] for i in indices])
    yb = torch.stack([data[i + 1: i + block_size + 1] for i in indices])
    return xb, yb

xb, yb = get_batch('train')
print(f'xb.shape = {xb.shape}')
print(f'xb = {xb}')
print(f'yb.shape = {yb.shape}')
print(f'yb = {yb}')

for i in range(batch_size):
    for j in range(block_size):
        print(f'when x = {xb[i][:j + 1]}, y = {yb[i][j]}')


xb.shape = torch.Size([4, 8])
xb = tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
yb.shape = torch.Size([4, 8])
yb = tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
when x = tensor([24]), y = 43
when x = tensor([24, 43]), y = 58
when x = tensor([24, 43, 58]), y = 5
when x = tensor([24, 43, 58,  5]), y = 57
when x = tensor([24, 43, 58,  5, 57]), y = 1
when x = tensor([24, 43, 58,  5, 57,  1]), y = 46
when x = tensor([24, 43, 58,  5, 57,  1, 46]), y = 43
when x = tensor([24, 43, 58,  5, 57,  1, 46, 43]), y = 39
when x = tensor([44]), y = 53
when x = tensor([44, 53]), y = 56
when x = tensor([44, 53, 56]), y = 1
when x = tensor([44, 53, 56,  1]), y = 58
when x = tensor([44, 53, 56,  1, 58]), y = 46
when x = tensor([44, 53, 56,  1, 58, 46]), y = 39
whe